Differential Expression Gene (DEG) Analysis Pipeline
====================================================
Author: Engki Nandatama  
Date: 2025-06-26  
Purpose: Comprehensive DEG analysis pipeline for portfolio project

This notebook performs end-to-end differential gene expression analysis
including data preprocessing, statistical testing, visualization, and
functional enrichment analysis.

Required Input Files:
1. gene_expression.tsv - Gene expression matrix (genes as rows, samples as columns)
2. metadata.tsv - Sample metadata with columns: 'SampleID' and 'Group'

Expected file structure:
- gene_expression.tsv: Tab-separated, first column = gene names/IDs
- metadata.tsv: Tab-separated with headers SampleID, Group (e.g., 'Tumor', 'Normal')

# =============================================================================
# CELL 1: Import Required Libraries and Setup
# =============================================================================

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, mannwhitneyu, shapiro
from statsmodels.stats.multitest import multipletests
import os
import warnings
from datetime import datetime
from gseapy import enrichr
import mygene

# Optional libraries (install if needed: pip install gseapy)
try:
    import gseapy as gp
    GSEA_AVAILABLE = True
except ImportError:
    print("Warning: gseapy not installed. GSEA analysis will be skipped.")
    GSEA_AVAILABLE = False

try:
    import mygene
except ImportError:
    print("Installing missing package: mygene...")
    !pip install mygene
    import mygene

# Setup plotting parameters
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Create output directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"DEG_analysis_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

print(f"🔬 DEG Analysis Pipeline Initialized")
print(f"📁 Output directory: {output_dir}")
print(f"📅 Analysis started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# =============================================================================
# CELL 2: Data Loading and Initial Validation
# =============================================================================

In [ ]:
def load_raw_data(expression_file, metadata_file):
    """
    Load raw gene expression matrix and metadata from TSV files.

    This function loads:
    1. The expression matrix (genes × samples) from a TSV file.
    2. The raw metadata file (e.g., TCGA-PRAD.clinical.tsv) containing various sample annotations.

    Note:
    - This function does not perform any validation of column names.
    - Metadata will be parsed in a separate step using a dedicated function.

    Parameters:
    -----------
    expression_file : str
        Path to gene expression matrix file (TSV format).
        Expected format: genes as rows, samples as columns.

    metadata_file : str
        Path to clinical/phenotype metadata file (TSV format).

    Returns:
    --------
    df_expr : pd.DataFrame
        Gene expression matrix (raw, unfiltered).

    df_meta_raw : pd.DataFrame
        Raw metadata dataframe to be parsed later.
    """
    print("📊 Loading data files...")

    # Load expression matrix
    try:
        df_expr = pd.read_csv(expression_file, sep='\t', index_col=0)
        print(f"✅ Expression data loaded: {df_expr.shape[0]} genes × {df_expr.shape[1]} samples")
    except FileNotFoundError:
        raise FileNotFoundError(f"❌ Expression file not found: {expression_file}")
    except Exception as e:
        raise Exception(f"❌ Error loading expression data: {str(e)}")

    # Load raw metadata
    try:
        df_meta_raw = pd.read_csv(metadata_file, sep='\t')
        print(f"✅ Metadata loaded: {df_meta_raw.shape[0]} samples")
    except FileNotFoundError:
        raise FileNotFoundError(f"❌ Metadata file not found: {metadata_file}")
    except Exception as e:
        raise Exception(f"❌ Error loading metadata: {str(e)}")

    return df_expr, df_meta_raw


# === MODIFY THESE FILE PATHS AS NEEDED ===
expression_file = "gene_expression.tsv"  # 👈 CHANGE THIS to your expression file
metadata_file = "metadata.tsv"  # 👈 CHANGE THIS to your metadata file
# ==========================================

# 🔽 Load the raw data files
df_expr, df_meta_raw = load_raw_data(expression_file, metadata_file)


# =============================================================================
# CELL 3: Data Preprocessing and Quality Check
# =============================================================================

In [ ]:
def parse_tcga_metadata(df_meta_raw):
    """
    Parse and simplify TCGA clinical metadata for DEG analysis.

    This function extracts relevant columns from the original metadata,
    renames them to standard identifiers ('SampleID', 'Group'),
    maps verbose group labels to simplified terms ('Tumor' or 'Normal'),
    and filters the metadata to include only these two groups.

    Parameters:
    -----------
    df_meta_raw : pd.DataFrame
        Raw clinical metadata downloaded from UCSC Xena
        (e.g., TCGA-PRAD.clinical.tsv)

    Returns:
    --------
    df_meta_clean : pd.DataFrame
        Cleaned metadata containing only 'SampleID' and 'Group' columns,
        suitable for downstream DEG analysis.
    """
    print("📄 Parsing original metadata...")

    # Select relevant columns: sample ID and sample type
    df_meta_clean = df_meta_raw[['sample', 'sample_type.samples']].copy()

    # Rename columns to match expected structure
    df_meta_clean.rename(columns={'sample': 'SampleID', 'sample_type.samples': 'Group'}, inplace=True)

    # Standardize group names
    df_meta_clean['Group'] = df_meta_clean['Group'].replace({
        'Primary Tumor': 'Tumor',
        'Solid Tissue Normal': 'Normal'
    })

    # Filter only the desired groups
    df_meta_clean = df_meta_clean[df_meta_clean['Group'].isin(['Tumor', 'Normal'])]

    # Show preview
    print(df_meta_clean.head())

    return df_meta_clean

print(df_meta_raw[['sample', 'sample_type.samples']].head())




def preprocess_expression_data(df_expr, df_meta):
    """
    Comprehensive preprocessing of gene expression data.

    Steps:
    1. Match samples between expression and metadata
    2. Remove genes with low/zero expression
    3. Log2 transformation
    4. Handle missing values

    Parameters:
    -----------
    df_expr : pd.DataFrame
        Raw gene expression matrix
    df_meta : pd.DataFrame
        Sample metadata

    Returns:
    --------
    df_expr_clean : pd.DataFrame
        Preprocessed expression matrix
    df_meta_clean : pd.DataFrame
        Matched metadata
    """

    print("🧹 Starting data preprocessing...")

    # Step 1: Match samples between expression data and metadata
    print("\n1️⃣ Matching samples between datasets...")
    valid_samples = df_meta['SampleID'].isin(df_expr.columns)
    df_meta_clean = df_meta[valid_samples].copy()

    # Keep only samples present in both datasets
    common_samples = df_meta_clean['SampleID'].tolist()
    df_expr_matched = df_expr[common_samples].copy()

    print(f"   • Samples in expression data: {df_expr.shape[1]}")
    print(f"   • Samples in metadata: {df_meta.shape[0]}")
    print(f"   • Common samples: {len(common_samples)}")

    # Step 2: Filter low-expressed genes
    print("\n2️⃣ Filtering low-expressed genes...")

    # Remove genes with all zero expression
    zero_genes = (df_expr_matched == 0).all(axis=1)
    print(f"   • Genes with all zeros: {zero_genes.sum()}")

    # Remove genes expressed in fewer than 3 samples (threshold > 1)
    min_samples = 3
    low_expr_genes = (df_expr_matched > 1).sum(axis=1) < min_samples
    print(f"   • Genes with low expression (<{min_samples} samples): {low_expr_genes.sum()}")

    # Apply filters
    genes_to_keep = ~(zero_genes | low_expr_genes)
    df_expr_filtered = df_expr_matched.loc[genes_to_keep].copy()

    print(f"   • Genes before filtering: {df_expr_matched.shape[0]}")
    print(f"   • Genes after filtering: {df_expr_filtered.shape[0]}")

    # Step 3: Log2 transformation (add pseudocount to avoid log(0))
    print("\n3️⃣ Applying log2 transformation...")
    df_expr_log = np.log2(df_expr_filtered + 1)

    # Step 4: Check for missing values
    print("\n4️⃣ Checking data quality...")
    missing_values = df_expr_log.isnull().sum().sum()
    print(f"   • Missing values after preprocessing: {missing_values}")

    if missing_values > 0:
        print("   ⚠️  Filling missing values with gene median...")
        df_expr_log = df_expr_log.fillna(df_expr_log.median(axis=1), axis=0)

    print(f"\n✅ Preprocessing complete!")
    print(f"   Final matrix shape: {df_expr_log.shape}")

    return df_expr_log, df_meta_clean

# Step: Parse metadata and preprocess expression matrix
df_meta_simple = parse_tcga_metadata(df_meta_raw)
df_expr_clean, df_meta_clean = preprocess_expression_data(df_expr, df_meta_simple)

# =============================================================================
# CELL 4: Exploratory Data Analysis and Visualization
# =============================================================================

In [ ]:
def exploratory_analysis(df_expr, df_meta, output_dir):
    """
    Perform exploratory data analysis with visualizations.

    Parameters:
    -----------
    df_expr : pd.DataFrame
        Preprocessed expression matrix
    df_meta : pd.DataFrame
        Sample metadata
    output_dir : str
        Directory to save plots
    """

    print("📈 Performing exploratory data analysis...")

    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Exploratory Data Analysis', fontsize=16, fontweight='bold')

    # Plot 1: Expression distribution
    axes[0,0].hist(df_expr.values.flatten(), bins=50, alpha=0.7, edgecolor='black')
    axes[0,0].set_title('Distribution of Log2 Expression Values')
    axes[0,0].set_xlabel('Log2(Expression + 1)')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].grid(True, alpha=0.3)

    # Plot 2: Sample-wise expression boxplot
    sample_means = df_expr.mean(axis=0)
    groups = df_meta.set_index('SampleID').loc[df_expr.columns]['Group']

    boxplot_data = [sample_means[groups == group] for group in groups.unique()]
    axes[0,1].boxplot(boxplot_data, labels=groups.unique())
    axes[0,1].set_title('Sample-wise Mean Expression by Group')
    axes[0,1].set_ylabel('Log2(Expression + 1)')
    axes[0,1].grid(True, alpha=0.3)

    # Plot 3: Sample correlation heatmap (subset for visibility)
    n_samples_to_show = min(20, df_expr.shape[1])
    corr_matrix = df_expr.iloc[:, :n_samples_to_show].corr()

    im = axes[1,0].imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
    axes[1,0].set_title(f'Sample Correlation Matrix (first {n_samples_to_show} samples)')
    axes[1,0].set_xlabel('Samples')
    axes[1,0].set_ylabel('Samples')
    plt.colorbar(im, ax=axes[1,0])

    # Plot 4: Group counts
    group_counts = df_meta['Group'].value_counts()
    axes[1,1].bar(group_counts.index, group_counts.values, alpha=0.7)
    axes[1,1].set_title('Sample Count by Group')
    axes[1,1].set_ylabel('Number of Samples')
    axes[1,1].grid(True, alpha=0.3)

    # Add count labels on bars
    for i, v in enumerate(group_counts.values):
        axes[1,1].text(i, v + 0.1, str(v), ha='center', va='bottom')

    plt.tight_layout()
    plt.savefig(f"{output_dir}/01_exploratory_analysis.png", dpi=300, bbox_inches='tight')
    plt.show()

    # Summary statistics
    print(f"\n📊 Data Summary Statistics:")
    print(f"   • Total genes: {df_expr.shape[0]:,}")
    print(f"   • Total samples: {df_expr.shape[1]}")
    print(f"   • Expression range: {df_expr.min().min():.2f} to {df_expr.max().max():.2f}")
    print(f"   • Mean expression: {df_expr.mean().mean():.2f}")
    print(f"   • Groups: {dict(group_counts)}")

# Perform exploratory analysis
exploratory_analysis(df_expr_clean, df_meta_clean, output_dir)


# =============================================================================
# CELL 5: Differential Expression Analysis
# =============================================================================

In [ ]:
def perform_deg_analysis(df_expr, df_meta, method='ttest', alpha=0.05, fc_threshold=1.0):
    """
    Perform differential gene expression analysis.

    Parameters:
    -----------
    df_expr : pd.DataFrame
        Gene expression matrix (log2 transformed)
    df_meta : pd.DataFrame
        Sample metadata
    method : str
        Statistical test method ('ttest' or 'mannwhitney')
    alpha : float
        Significance threshold for p-values
    fc_threshold : float
        Log2 fold change threshold

    Returns:
    --------
    results_df : pd.DataFrame
        Complete DEG analysis results
    """

    print(f"🧬 Performing DEG analysis using {method}...")

    # Get group information
    groups = df_meta.set_index('SampleID').loc[df_expr.columns]['Group']
    unique_groups = groups.unique()

    if len(unique_groups) != 2:
        raise ValueError(f"❌ Expected exactly 2 groups, found {len(unique_groups)}: {unique_groups}")

    group1_name, group2_name = unique_groups[0], unique_groups[1]
    group1_samples = df_expr.loc[:, groups == group1_name]
    group2_samples = df_expr.loc[:, groups == group2_name]

    print(f"   • Group 1 ({group1_name}): {group1_samples.shape[1]} samples")
    print(f"   • Group 2 ({group2_name}): {group2_samples.shape[1]} samples")
    print(f"   • Testing {df_expr.shape[0]:,} genes...")

    # Perform statistical testing
    results = []

    for i, gene in enumerate(df_expr.index):
        if (i + 1) % 1000 == 0:  # Progress indicator
            print(f"   Processing gene {i+1:,}/{df_expr.shape[0]:,}...")

        group1_values = group1_samples.loc[gene].values
        group2_values = group2_samples.loc[gene].values

        # Calculate means and fold change
        mean1 = np.mean(group1_values)
        mean2 = np.mean(group2_values)

        # Calculate log2 fold change (Group2 vs Group1)
        # Add small pseudocount to avoid division by zero
        fold_change = (mean2 + 1e-8) / (mean1 + 1e-8)
        log2fc = np.log2(fold_change)

        # Statistical test
        try:
            if method == 'ttest':
                stat, pval = ttest_ind(group2_values, group1_values, equal_var=False)
            elif method == 'mannwhitney':
                stat, pval = mannwhitneyu(group2_values, group1_values, alternative='two-sided')
            else:
                raise ValueError(f"Unknown method: {method}")
        except:
            # Handle edge cases (e.g., constant values)
            stat, pval = np.nan, 1.0

        results.append({
            'Gene': gene,
            'Group1_mean': mean1,
            'Group2_mean': mean2,
            'log2FC': log2fc,
            'stat': stat,
            'p_value': pval,
            'Group1_name': group1_name,
            'Group2_name': group2_name
        })

    # Create results dataframe
    results_df = pd.DataFrame(results)

    # Multiple testing correction using Benjamini-Hochberg (FDR)
    print(f"\n🔧 Applying multiple testing correction...")
    valid_pvals = ~np.isnan(results_df['p_value'])

    if valid_pvals.sum() > 0:
        _, adj_pvals, _, _ = multipletests(
            results_df.loc[valid_pvals, 'p_value'],
            method='fdr_bh'
        )
        results_df.loc[valid_pvals, 'adj_p_value'] = adj_pvals
        results_df.loc[~valid_pvals, 'adj_p_value'] = 1.0
    else:
        results_df['adj_p_value'] = 1.0

    # Add significance flags
    results_df['significant'] = (
        (results_df['adj_p_value'] < alpha) &
        (np.abs(results_df['log2FC']) > fc_threshold)
    )

    # Add regulation direction
    results_df['regulation'] = 'Not Significant'
    results_df.loc[
        (results_df['significant']) & (results_df['log2FC'] > 0),
        'regulation'
    ] = f'Up in {group2_name}'
    results_df.loc[
        (results_df['significant']) & (results_df['log2FC'] < 0),
        'regulation'
    ] = f'Down in {group2_name}'

    # Sort by adjusted p-value
    results_df = results_df.sort_values('adj_p_value').reset_index(drop=True)

    # Summary statistics
    n_significant = results_df['significant'].sum()
    n_upregulated = (results_df['regulation'] == f'Up in {group2_name}').sum()
    n_downregulated = (results_df['regulation'] == f'Down in {group2_name}').sum()

    print(f"\n📊 DEG Analysis Results:")
    print(f"   • Total genes tested: {len(results_df):,}")
    print(f"   • Significant genes (adj p < {alpha}, |log2FC| > {fc_threshold}): {n_significant:,}")
    print(f"   • Upregulated in {group2_name}: {n_upregulated:,}")
    print(f"   • Downregulated in {group2_name}: {n_downregulated:,}")

    return results_df

# Perform DEG analysis
# You can change method to 'mannwhitney' for non-parametric test
deg_results = perform_deg_analysis(
    df_expr_clean,
    df_meta_clean,
    method='ttest',        # 👈 CHANGE to 'mannwhitney' if data is not normally distributed
    alpha=0.05,           # 👈 CHANGE significance threshold if needed
    fc_threshold=1.0      # 👈 CHANGE fold change threshold if needed
)

# Save results
deg_results.to_csv(f"{output_dir}/02_DEG_results_complete.tsv", sep='\t', index=False)
significant_genes = deg_results[deg_results['significant']]
significant_genes.to_csv(f"{output_dir}/03_DEG_significant_genes.tsv", sep='\t', index=False)

print(f"\n💾 Results saved:")
print(f"   • Complete results: 02_DEG_results_complete.tsv")
print(f"   • Significant genes: 03_DEG_significant_genes.tsv")

# =============================================================================
# CELL 6: Volcano Plot Visualization
# =============================================================================

In [ ]:
def create_volcano_plot(results_df, output_dir, alpha=0.05, fc_threshold=1.0,
                       top_genes=10, figsize=(12, 8)):
    """
    Create an enhanced volcano plot for DEG results.

    Parameters:
    -----------
    results_df : pd.DataFrame
        DEG analysis results
    output_dir : str
        Directory to save the plot
    alpha : float
        Significance threshold
    fc_threshold : float
        Fold change threshold
    top_genes : int
        Number of top genes to label
    figsize : tuple
        Figure size
    """

    print("🌋 Creating volcano plot...")

    fig, ax = plt.subplots(figsize=figsize)

    # Calculate -log10(p-value) for y-axis
    results_df['-log10_pval'] = -np.log10(results_df['adj_p_value'].replace(0, 1e-300))

    # Define colors for different categories
    colors = {
        'Not Significant': '#CCCCCC',
        f'Up in {results_df["Group2_name"].iloc[0]}': '#FF6B6B',
        f'Down in {results_df["Group2_name"].iloc[0]}': '#4ECDC4'
    }

    # Create scatter plot for each category
    for regulation in colors.keys():
        subset = results_df[results_df['regulation'] == regulation]
        ax.scatter(
            subset['log2FC'],
            subset['-log10_pval'],
            c=colors[regulation],
            alpha=0.6,
            s=30,
            label=f"{regulation} (n={len(subset)})",
            edgecolors='none'
        )

    # Add threshold lines
    ax.axhline(y=-np.log10(alpha), color='blue', linestyle='--', alpha=0.7,
               label=f'adj p-value = {alpha}')
    ax.axvline(x=fc_threshold, color='blue', linestyle='--', alpha=0.7)
    ax.axvline(x=-fc_threshold, color='blue', linestyle='--', alpha=0.7,
               label=f'|log2FC| = {fc_threshold}')

    # Label top significant genes
    significant_genes = results_df[results_df['significant']].head(top_genes)

    for _, gene_data in significant_genes.iterrows():
        ax.annotate(
            gene_data['Gene'],
            (gene_data['log2FC'], gene_data['-log10_pval']),
            xytext=(5, 5),
            textcoords='offset points',
            fontsize=8,
            alpha=0.8,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7)
        )

    # Customize plot
    ax.set_xlabel('log2(Fold Change)', fontsize=12, fontweight='bold')
    ax.set_ylabel('-log10(Adjusted P-value)', fontsize=12, fontweight='bold')
    ax.set_title('Volcano Plot - Differential Gene Expression', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    # Add statistics text box
    n_total = len(results_df)
    n_sig = results_df['significant'].sum()
    n_up = (results_df['regulation'] == f'Up in {results_df["Group2_name"].iloc[0]}').sum()
    n_down = (results_df['regulation'] == f'Down in {results_df["Group2_name"].iloc[0]}').sum()

    stats_text = f"""Statistics:
Total genes: {n_total:,}
Significant: {n_sig:,} ({n_sig/n_total*100:.1f}%)
Upregulated: {n_up:,}
Downregulated: {n_down:,}"""

    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

    plt.tight_layout()
    plt.savefig(f"{output_dir}/04_volcano_plot.png", dpi=300, bbox_inches='tight')
    plt.show()

    print(f"✅ Volcano plot saved: 04_volcano_plot.png")

# Create volcano plot
create_volcano_plot(deg_results, output_dir)

# =============================================================================
# CELL 7: Heatmap of Top Differentially Expressed Genes
# =============================================================================

In [ ]:
def create_expression_heatmap(df_expr, deg_results, df_meta, output_dir,
                            top_n=50, figsize=(12, 10)):
    """
    Create clustered heatmap of top differentially expressed genes.

    Parameters:
    -----------
    df_expr : pd.DataFrame
        Gene expression matrix
    deg_results : pd.DataFrame
        DEG analysis results
    df_meta : pd.DataFrame
        Sample metadata
    output_dir : str
        Directory to save the plot
    top_n : int
        Number of top genes to include
    figsize : tuple
        Figure size
    """

    print(f"🔥 Creating heatmap of top {top_n} differentially expressed genes...")

    # Get top significant genes
    top_genes = deg_results[deg_results['significant']].head(top_n)['Gene'].tolist()

    if len(top_genes) == 0:
        print("⚠️  No significant genes found for heatmap!")
        return

    if len(top_genes) < top_n:
        print(f"⚠️  Only {len(top_genes)} significant genes available (requested {top_n})")

    # Extract expression data for top genes
    heatmap_data = df_expr.loc[top_genes].copy()

    # Create sample annotation for heatmap
    sample_groups = df_meta.set_index('SampleID').loc[heatmap_data.columns]['Group']

    # Create annotation colors
    unique_groups = sample_groups.unique()
    group_colors = sns.color_palette("husl", len(unique_groups))
    color_map = dict(zip(unique_groups, group_colors))
    sample_colors = [color_map[group] for group in sample_groups]

    # Create the clustered heatmap
    plt.figure(figsize=figsize)

    # Use clustermap for hierarchical clustering
    g = sns.clustermap(
        heatmap_data,
        cmap='RdBu_r',
        center=0,
        z_score=0,  # Standardize by row (gene)
        col_colors=sample_colors,
        figsize=figsize,
        dendrogram_ratio=0.1,
        colors_ratio=0.03,
        cbar_pos=(0.02, 0.8, 0.03, 0.18)
    )

    # Customize the plot
    g.ax_heatmap.set_xlabel('Samples', fontsize=12, fontweight='bold')
    g.ax_heatmap.set_ylabel('Genes', fontsize=12, fontweight='bold')
    g.fig.suptitle(f'Heatmap of Top {len(top_genes)} Differentially Expressed Genes',
                   fontsize=14, fontweight='bold', y=0.98)

    # Create legend for sample groups
    legend_elements = [plt.Rectangle((0,0),1,1, facecolor=color_map[group],
                                   label=group) for group in unique_groups]
    g.ax_heatmap.legend(handles=legend_elements, title='Sample Groups',
                       bbox_to_anchor=(1.15, 1), loc='upper left')

    plt.savefig(f"{output_dir}/05_heatmap_top_genes.png", dpi=300, bbox_inches='tight')
    plt.show()

    print(f"✅ Heatmap saved: 05_heatmap_top_genes.png")

    # Save gene list used in heatmap
    heatmap_genes_df = deg_results[deg_results['Gene'].isin(top_genes)][
        ['Gene', 'log2FC', 'adj_p_value', 'regulation']
    ]
    heatmap_genes_df.to_csv(f"{output_dir}/06_heatmap_gene_list.tsv", sep='\t', index=False)
    print(f"✅ Heatmap gene list saved: 06_heatmap_gene_list.tsv")

# Create heatmap
create_expression_heatmap(df_expr_clean, deg_results, df_meta_clean, output_dir, top_n=50)

# =============================================================================
# CELL 8: Gene Set Enrichment Analysis (Optional)
# =============================================================================

In [ ]:
# === Function 1: Create Summary Plot ===
def create_enrichment_summary_plot(enrichment_results, output_dir):
    """
    Create a summary barplot of top enriched terms across multiple gene sets.

    Parameters:
    -----------
    enrichment_results : dict
        Dictionary of enrichment DataFrames, one per gene set.

    output_dir : str
        Directory to save the plot.
    """
    print("📊 Creating enrichment summary plot...")

    all_terms = []

    # Combine top 5 terms from each gene set
    for gene_set, df in enrichment_results.items():
        if not df.empty:
            top_terms = df.head(5).copy()
            top_terms['Gene_Set'] = gene_set
            all_terms.append(top_terms)

    if not all_terms:
        print("⚠️ No enrichment results to plot.")
        return

    combined = pd.concat(all_terms, ignore_index=True)
    combined['-log10(Adj.P)'] = -np.log10(combined['Adjusted P-value'])

    # Sort and plot top 20 terms
    plot_data = combined.sort_values('Adjusted P-value').head(20)

    plt.figure(figsize=(12, 8))
    sns.barplot(
        y='Term',
        x='-log10(Adj.P)',
        hue='Gene_Set',
        data=plot_data,
        dodge=False
    )
    plt.title("Top Enriched Terms Across Gene Sets", fontsize=14, weight='bold')
    plt.xlabel("-log10(Adjusted P-value)", fontsize=12)
    plt.ylabel("")
    plt.legend(title='Gene Set')
    plt.tight_layout()

    plot_path = os.path.join(output_dir, "07_enrichment_summary.png")
    plt.savefig(plot_path, dpi=300)
    plt.show()

    print(f"✅ Enrichment summary plot saved: {plot_path}")

# === Function 2: Perform GSEA Analysis ===
def perform_gsea_analysis(deg_results, output_dir, organism='Human', gene_sets=None):
    """
    Perform Gene Set Enrichment Analysis (GSEA) using Enrichr via gseapy,
    mapping Ensembl gene IDs to gene symbols and filtering for protein-coding genes.

    Parameters:
    -----------
    deg_results : pd.DataFrame
        DataFrame containing DEG results with columns:
        - 'Gene' (Ensembl ID)
        - 'significant' (boolean flag)

    output_dir : str
        Directory to save enrichment results

    organism : str, default='Human'
        Target organism for enrichment libraries (used by Enrichr)

    gene_sets : list of str, optional
        List of Enrichr gene set libraries to use
    """
    if not 'enrichr' in globals():
        print("⚠️ GSEA skipped – gseapy not available. Install with: pip install gseapy mygene")
        return

    print("🧬 Starting Gene Set Enrichment Analysis...")

    # Default gene sets
    if gene_sets is None:
        gene_sets = [
            'GO_Biological_Process_2023',
            'GO_Cellular_Component_2023',
            'GO_Molecular_Function_2023',
            'KEGG_2021_Human'
        ]

    # Step 1: Get significant Ensembl IDs
    ensembl_ids = deg_results[deg_results['significant']]['Gene'].dropna().tolist()
    if len(ensembl_ids) < 10:
        print(f"⚠️ Too few significant genes ({len(ensembl_ids)}) for GSEA.")
        return

    # Step 2: Remove version suffixes (e.g., ".5")
    ensembl_ids_clean = [gene.split('.')[0] for gene in ensembl_ids]

    # Step 3: Map to gene symbols and filter only protein-coding
    print("🔄 Mapping Ensembl IDs to protein-coding gene symbols...")
    mg = mygene.MyGeneInfo()
    query_results = mg.querymany(
        ensembl_ids_clean,
        scopes='ensembl.gene',
        fields=['symbol', 'type_of_gene'],
        species='human',
        as_dataframe=True
    )

    if 'symbol' not in query_results.columns:
        print("❌ Mapping failed: no symbols found.")
        return

    # Filter only protein-coding genes
    filtered = query_results[
        (query_results['type_of_gene'] == 'protein-coding') &
        (query_results['symbol'].notnull())
    ]
    gene_symbols = filtered['symbol'].unique().tolist()

    if len(gene_symbols) < 10:
        print(f"❌ Too few protein-coding gene symbols ({len(gene_symbols)}) mapped for enrichment.")
        return

    print(f"   • Mapped gene symbols (protein-coding): {len(gene_symbols)}")
    print(f"   • Gene sets: {gene_sets}")

    # Step 4: Prepare output folder
    gsea_dir = os.path.join(output_dir, "GSEA_results")
    os.makedirs(gsea_dir, exist_ok=True)

    enrichment_results = {}

    # Step 5: Enrichment per gene set
    for gene_set in gene_sets:
        print(f"\n📚 Enriching gene set: {gene_set}")
        try:
            enr = enrichr(
                gene_list=gene_symbols,
                gene_sets=[gene_set],
                organism=organism,
                outdir=os.path.join(gsea_dir, gene_set),
                cutoff=0.1,
                no_plot=False
            )
            if len(enr.results) > 0:
                result_path = os.path.join(gsea_dir, f"{gene_set}_enrichment.tsv")
                enr.results.to_csv(result_path, sep='\t', index=False)
                enrichment_results[gene_set] = enr.results.head(10)
                print(f"     ✅ {len(enr.results)} enriched terms found")
            else:
                print("     ❌ No significant enrichment found")
        except Exception as e:
            print(f"     ❌ Error analyzing {gene_set}: {str(e)}")

    # Step 6: Summary plot
    if enrichment_results:
        create_enrichment_summary_plot(enrichment_results, output_dir)

    print(f"\n✅ GSEA analysis complete. Results saved to: {gsea_dir}")

# === Run the Analysis ===
# Make sure deg_results and output_dir are already defined from previous cells
perform_gsea_analysis(deg_results, output_dir)


# =============================================================================
# CELL 9: GSEA Result Tables (Top Enriched Terms)
# =============================================================================

In [ ]:
def show_gsea_table(gsea_dir, gene_set, top_n=20):
    """
    Display top enriched terms from a GSEA result file.

    Parameters:
    -----------
    gsea_dir : str
        Path to GSEA_results folder
    gene_set : str
        Specific gene set name
    top_n : int
        Number of top terms to display
    """
    path = os.path.join(gsea_dir, f"{gene_set}_enrichment.tsv")
    if not os.path.exists(path):
        print(f"⚠️ File not found: {path}")
        return

    df = pd.read_csv(path, sep="\t").head(top_n)
    display(df)

# === Show Tables ===
gsea_dir = os.path.join(output_dir, "GSEA_results")

print("\n📝 Top 20 Enriched Terms from GO Biological Process")
show_gsea_table(gsea_dir, 'GO_Biological_Process_2023', top_n=20)

print("\n📝 Top 20 Enriched Terms from KEGG Pathways")
show_gsea_table(gsea_dir, 'KEGG_2021_Human', top_n=20)


# =============================================================================
# CELL 10: Statistical Summary and Quality Metrics
# =============================================================================

In [ ]:
def generate_analysis_summary(deg_results, df_expr, df_meta, output_dir):
    """
    Generate comprehensive analysis summary with quality metrics.

    Parameters:
    -----------
    deg_results : pd.DataFrame
        DEG analysis results
    df_expr : pd.DataFrame
        Expression data
    df_meta : pd.DataFrame
        Metadata
    output_dir : str
        Output directory
    """

    print("📋 Generating analysis summary...")

    # Basic statistics
    n_genes_total = len(deg_results)
    n_genes_sig = deg_results['significant'].sum()
    n_upregulated = (deg_results['regulation'].str.contains('Up')).sum()
    n_downregulated = (deg_results['regulation'].str.contains('Down')).sum()

    group_names = deg_results[['Group1_name', 'Group2_name']].iloc[0]
    group_counts = df_meta['Group'].value_counts()

    # p-value distribution
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Analysis Summary and Quality Metrics', fontsize=16, fontweight='bold')

    # Plot 1: P-value distribution
    axes[0,0].hist(deg_results['p_value'].dropna(), bins=50, alpha=0.7, edgecolor='black')
    axes[0,0].set_title('P-value Distribution')
    axes[0,0].set_xlabel('P-value')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].axvline(x=0.05, color='red', linestyle='--', label='α = 0.05')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)

    # Plot 2: Fold change distribution
    axes[0,1].hist(deg_results['log2FC'].dropna(), bins=50, alpha=0.7, edgecolor='black')
    axes[0,1].set_title('Log2 Fold Change Distribution')
    axes[0,1].set_xlabel('Log2 Fold Change')
    axes[0,1].set_ylabel('Frequency')
    axes[0,1].axvline(x=1, color='red', linestyle='--', label='FC = 2')
    axes[0,1].axvline(x=-1, color='red', linestyle='--', label='FC = 0.5')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)

    # Plot 3: MA plot (Mean vs Fold Change)
    mean_expr = (deg_results['Group1_mean'] + deg_results['Group2_mean']) / 2
    axes[1,0].scatter(mean_expr, deg_results['log2FC'], alpha=0.5, s=10)
    axes[1,0].set_title('MA Plot')
    axes[1,0].set_xlabel('Mean Expression')
    axes[1,0].set_ylabel('Log2 Fold Change')
    axes[1,0].axhline(y=0, color='red', linestyle='-', alpha=0.7)
    axes[1,0].grid(True, alpha=0.3)

    # Plot 4: Top regulated genes
    top_genes = deg_results[deg_results['significant']].head(10)
    if len(top_genes) > 0:
        bars = axes[1,1].barh(range(len(top_genes)), top_genes['log2FC'],
                             color=['red' if fc > 0 else 'blue' for fc in top_genes['log2FC']])
        axes[1,1].set_yticks(range(len(top_genes)))
        axes[1,1].set_yticklabels(top_genes['Gene'], fontsize=8)
        axes[1,1].set_title('Top 10 Differentially Expressed Genes')
        axes[1,1].set_xlabel('Log2 Fold Change')
        axes[1,1].grid(True, alpha=0.3)
    else:
        axes[1,1].text(0.5, 0.5, 'No significant genes found',
                      ha='center', va='center', transform=axes[1,1].transAxes)
        axes[1,1].set_title('Top Differentially Expressed Genes')

    plt.tight_layout()
    plt.savefig(f"{output_dir}/08_analysis_summary.png", dpi=300, bbox_inches='tight')
    plt.show()

    # Generate text summary
    summary_text = f"""
DIFFERENTIAL GENE EXPRESSION ANALYSIS SUMMARY
=============================================

Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

DATASET INFORMATION:
- Total genes analyzed: {n_genes_total:,}
- Total samples: {df_expr.shape[1]}
- Groups compared: {group_names['Group1_name']} (n={group_counts[group_names['Group1_name']]}) vs {group_names['Group2_name']} (n={group_counts[group_names['Group2_name']]})

RESULTS SUMMARY:
- Significant genes (adj p < 0.05, |log2FC| > 1.0): {n_genes_sig:,} ({n_genes_sig/n_genes_total*100:.2f}%)
- Upregulated in {group_names['Group2_name']}: {n_upregulated:,} ({n_upregulated/n_genes_total*100:.2f}%)
- Downregulated in {group_names['Group2_name']}: {n_downregulated:,} ({n_downregulated/n_genes_total*100:.2f}%)

TOP 10 UPREGULATED GENES:
"""

    # Add top upregulated genes
    top_up = deg_results[
        (deg_results['regulation'].str.contains('Up')) &
        (deg_results['significant'])
    ].head(10)

    if len(top_up) > 0:
        for _, gene in top_up.iterrows():
            summary_text += f"- {gene['Gene']}: log2FC = {gene['log2FC']:.2f}, adj p = {gene['adj_p_value']:.2e}\n"
    else:
        summary_text += "- No significantly upregulated genes found\n"

    summary_text += f"\nTOP 10 DOWNREGULATED GENES:\n"

    # Add top downregulated genes
    top_down = deg_results[
        (deg_results['regulation'].str.contains('Down')) &
        (deg_results['significant'])
    ].head(10)

    if len(top_down) > 0:
        for _, gene in top_down.iterrows():
            summary_text += f"- {gene['Gene']}: log2FC = {gene['log2FC']:.2f}, adj p = {gene['adj_p_value']:.2e}\n"
    else:
        summary_text += "- No significantly downregulated genes found\n"

    summary_text += f"""
QUALITY METRICS:
- Mean p-value: {deg_results['p_value'].mean():.4f}
- Median p-value: {deg_results['p_value'].median():.4f}
- Mean |log2FC|: {np.abs(deg_results['log2FC']).mean():.4f}
- Max |log2FC|: {np.abs(deg_results['log2FC']).max():.4f}

FILES GENERATED:
- 01_exploratory_analysis.png: Initial data exploration
- 02_DEG_results_complete.tsv: Complete DEG analysis results
- 03_DEG_significant_genes.tsv: Significant genes only
- 04_volcano_plot.png: Volcano plot visualization
- 05_heatmap_top_genes.png: Heatmap of top genes
- 06_heatmap_gene_list.tsv: Genes used in heatmap
- 07_enrichment_summary.png: GSEA results (if available)
- 08_analysis_summary.png: Summary plots and metrics
- 09_analysis_summary.txt: This summary report

INTERPRETATION NOTES:
- Positive log2FC indicates higher expression in {group_names['Group2_name']}
- Negative log2FC indicates higher expression in {group_names['Group1_name']}
- Adjusted p-values control for multiple testing using Benjamini-Hochberg FDR
- Consider validating top genes with independent methods (qPCR, Western blot)
"""

    # Save summary
    with open(f"{output_dir}/09_analysis_summary.txt", 'w') as f:
        f.write(summary_text)

    print(f"✅ Analysis summary saved: 09_analysis_summary.txt")
    print(f"\n{summary_text}")

# Generate analysis summary
generate_analysis_summary(deg_results, df_expr_clean, df_meta_clean, output_dir)

# =============================================================================
# CELL 11: Export Results for Further Analysis
# =============================================================================

In [ ]:
def export_results_for_downstream(deg_results, df_expr, df_meta, output_dir):
    """
    Export results in formats suitable for downstream analysis.

    Parameters:
    -----------
    deg_results : pd.DataFrame
        DEG analysis results
    df_expr : pd.DataFrame
        Expression data
    df_meta : pd.DataFrame
        Metadata
    output_dir : str
        Output directory
    """

    print("💾 Exporting results for downstream analysis...")

    # 1. Export gene lists for pathway analysis tools
    significant_genes = deg_results[deg_results['significant']]

    # All significant genes
    with open(f"{output_dir}/gene_lists_all_significant.txt", 'w') as f:
        f.write('\n'.join(significant_genes['Gene'].tolist()))

    # Upregulated genes only
    upregulated = significant_genes[significant_genes['log2FC'] > 0]
    with open(f"{output_dir}/gene_lists_upregulated.txt", 'w') as f:
        f.write('\n'.join(upregulated['Gene'].tolist()))

    # Downregulated genes only
    downregulated = significant_genes[significant_genes['log2FC'] < 0]
    with open(f"{output_dir}/gene_lists_downregulated.txt", 'w') as f:
        f.write('\n'.join(downregulated['Gene'].tolist()))

    # 2. Export ranked gene list for GSEA
    ranked_genes = deg_results[['Gene', 'log2FC']].copy()
    ranked_genes = ranked_genes.sort_values('log2FC', ascending=False)
    ranked_genes.to_csv(f"{output_dir}/ranked_gene_list.rnk", sep='\t', index=False, header=False)

    # 3. Export expression data for significant genes
    if len(significant_genes) > 0:
        sig_expr = df_expr.loc[significant_genes['Gene']]
        sig_expr.to_csv(f"{output_dir}/expression_significant_genes.tsv", sep='\t')

    # 4. Export sample metadata
    df_meta.to_csv(f"{output_dir}/sample_metadata.tsv", sep='\t', index=False)

    # 5. Create Excel file with multiple sheets
    try:
        with pd.ExcelWriter(f"{output_dir}/DEG_analysis_complete.xlsx", engine='openpyxl') as writer:
            deg_results.to_excel(writer, sheet_name='All_Results', index=False)
            significant_genes.to_excel(writer, sheet_name='Significant_Genes', index=False)
            upregulated.to_excel(writer, sheet_name='Upregulated', index=False)
            downregulated.to_excel(writer, sheet_name='Downregulated', index=False)
            df_meta.to_excel(writer, sheet_name='Sample_Metadata', index=False)

        print("✅ Excel file created: DEG_analysis_complete.xlsx")
    except ImportError:
        print("⚠️  Excel export skipped - openpyxl not installed")

    print(f"""
📁 Files exported for downstream analysis:
   • gene_lists_all_significant.txt - All significant genes
   • gene_lists_upregulated.txt - Upregulated genes only
   • gene_lists_downregulated.txt - Downregulated genes only
   • ranked_gene_list.rnk - Ranked gene list for GSEA
   • expression_significant_genes.tsv - Expression data for significant genes
   • sample_metadata.tsv - Sample information
   • DEG_analysis_complete.xlsx - Complete results in Excel format
""")

# Export results
export_results_for_downstream(deg_results, df_expr_clean, df_meta_clean, output_dir)


# =============================================================================
# CELL 12: Final Summary and Next Steps
# =============================================================================

In [ ]:
from datetime import datetime

print("\n🎉 DIFFERENTIAL GENE EXPRESSION ANALYSIS COMPLETED!")
print("=" * 60)

# Final DEG statistics
n_total = len(deg_results)
n_significant = deg_results['significant'].sum()
n_up = (deg_results['regulation'].str.contains('Up')).sum()
n_down = (deg_results['regulation'].str.contains('Down')).sum()

# Terminal summary
print(f"""
📊 FINAL DEG RESULTS:
   • Total genes analyzed      : {n_total:,}
   • Significant DEGs found    : {n_significant:,} ({n_significant/n_total*100:.2f}%)
   • Upregulated genes         : {n_up:,}
   • Downregulated genes       : {n_down:,}
""")

# GSEA summary
gsea_dir = os.path.join(output_dir, "GSEA_results")
gsea_summary_lines = []
if os.path.exists(gsea_dir):
    print("🧬 GSEA Summary:")
    for fname in os.listdir(gsea_dir):
        if fname.endswith("_enrichment.tsv"):
            path = os.path.join(gsea_dir, fname)
            df = pd.read_csv(path, sep="\t")
            enriched_count = (df['Adjusted P-value'] < 0.05).sum()
            summary_line = f"- **{fname.replace('_enrichment.tsv','')}**: {enriched_count} enriched terms"
            gsea_summary_lines.append(summary_line)
            print(f"   • {fname.replace('_enrichment.tsv','')}: {enriched_count} enriched terms")
else:
    print("⚠️  No GSEA directory found — skipping enrichment summary.")
    gsea_summary_lines.append("- No GSEA enrichment results available.")

# Join lines into markdown-compatible text
gsea_text = "\n".join(gsea_summary_lines)

# Markdown report content
report_content = f"""
# 📊 Differential Gene Expression Analysis Report

**Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## 📌 Summary
- Total genes analyzed     : {n_total:,}
- Significant DEGs         : {n_significant:,}
- Upregulated genes        : {n_up:,}
- Downregulated genes      : {n_down:,}

## 🧬 GSEA Enrichment Summary
{gsea_text}

## 📁 Files Generated
All results, visualizations, and exports are saved to the folder: `{output_dir}`

- DEG tables, gene lists, ranked genes
- Expression matrix (filtered)
- Enrichment analysis and plots
- Project summary report

## 🧪 Methods and Tools Used
- **Data Source**: UCSC Xena (TCGA RNA-Seq and clinical metadata) --example
- **Statistical Test**: Welch’s t-test (unequal variance)
- **Correction**: Benjamini-Hochberg FDR
- **Threshold**: Adjusted p-value < 0.05 and |log2FC| > 1
- **Visualization**: Volcano plot, heatmap, GSEA summary
- **GSEA**: Enrichr via GSEApy
- **Annotation**: MyGene.info
- **Python**: Version 3.11, Jupyter/Colab ready

## 💡 Next Steps
1. **Biological Interpretation**
   - Research top DEGs and their biological roles
   - Link to known pathways or mechanisms

2. **Validation**
   - Design qPCR validation
   - Compare with published studies

3. **Further Analysis**
   - Pathway/network analysis (KEGG, GO, Reactome)
   - Enrichment tools like STRINGdb, DAVID, g:Profiler

4. **Portfolio Enhancement**
   - Include plots and interpretation
   - Describe methodology and decisions
   - Compare with other tools or thresholds

## ⚖️ License and Usage
This notebook is free to use under the **MIT License**.
Attribution is appreciated.

## 📚 Citation
If this notebook helped your project, please cite:

> Nandatama, E. (2025). *DEG Analysis Pipeline in Python*. GitHub: [your-link]

Also cite:
- **GSEApy**: Fang et al. Bioinformatics (2021)
- **MyGene.info**: BioThings API

## ⚠️ Disclaimer
Note: This pipeline is intended for research and educational purposes only. The findings should not be interpreted as clinical advice or used to guide patient care without further experimental validation.

---
*Analysis completed on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*
"""

# Save to file
with open(f"{output_dir}/PROJECT_REPORT.md", "w") as f:
    f.write(report_content)

print(f"✅ Project report saved to: {output_dir}/PROJECT_REPORT.md")
print("🚀 Your DEG pipeline is complete and ready for portfolio or publication use!")